## Test 1
This notebook should fine tune the models for test one

**Installation**


In [1]:
resume_training = False

In [2]:
print("---Installing Dependecies---")
import os, re # Import OS and Regex tools
import torch; # Install torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0) # Version for xformers
xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
!python3 -m pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo # Install other ML dependencies
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer # Install huggingface API
!pip install --no-deps unsloth # Install Unsloth. We use this to speed up training
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install "numpy<2.0"
!python -m xformers.info

---Installing Dependecies---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall

**Model settings**

In [3]:
print("---Initialising model---")

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = torch.float16
load_in_4bit = True # 4bit loading. This optimization saves us memory
if not resume_training:
    print("Loading fresh model")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    ) # Returns a tuple with model (actaul neural network) and tokeiser (text processor)
else:
    print("Loading trained model")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/kaggle/input/trained/other/default/1/trained",     # SAVED CHECKPOINT
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

---Initialising model---
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-11 19:25:11.702847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765481112.139696      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765481112.269805      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


[unsloth_zoo.log|WARNING]Unsloth: Failed to import trl openenv: No module named 'trl.experimental'


Loading fresh model
==((====))==  Unsloth 2025.12.4: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

**Fine tuning settings**

In [4]:
if not resume_training:
    print("Applying Adapters")
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 32, # Scale factor. Larger values make the model learn quicker
        lora_dropout = 0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 13, # Random number, keep at 0013 for reproducability
        use_rslora = False,
        loftq_config = None,
    ) # This takes the base model as input and over writes the variabe with the model but with LoRa adapters onto it
else: 
    print("Skipping PEFT adapters since training resuming")

Applying Adapters
Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


**Data Preperation**


In [5]:
print("---Loading Data---")
from datasets import load_dataset

dataset = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', 'en', split = "train")
print(dataset.column_names)

---Loading Data---


README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

['Question', 'Complex_CoT', 'Response']


In [6]:
dataset.column_names

['Question', 'Complex_CoT', 'Response']

In [7]:
from datasets import concatenate_datasets
d_full, unused = dataset.train_test_split(test_size = 0.9).values()

print(f"Full dataset: {d_full.num_rows} rows")

d_test_subtopic = d_full.filter(
    lambda x: "fever" in x["Question"].lower()
) # Everything that has cancer mentioned in response
print(f"Subtopic: {d_test_subtopic.num_rows}")

d_test_NOTsubtopic = d_full.filter(
    lambda x: "fever" not in x["Question"].lower()
)
print(f"NOT Subtopic: {d_test_NOTsubtopic.num_rows}")

# Since this is the control model, however if we train it and then test it on the same info it will be pointless so we remove half of the rows abput the subtopic for testing

d_train, d_test_NOTsubtopic = d_test_NOTsubtopic.train_test_split(test_size=0.2, seed=13).values()
temp, d_test_subtopic = d_test_subtopic.train_test_split(test_size=0.2, seed=13).values()

d_train = concatenate_datasets([d_train, temp])

print(f"Train dataset: {d_train.num_rows}")
print(f"Subtopic test: {d_test_subtopic.num_rows}")
print(f"NOT Subtopic test: {d_test_NOTsubtopic.num_rows}")


Full dataset: 1970 rows


Filter:   0%|          | 0/1970 [00:00<?, ? examples/s]

Subtopic: 153


Filter:   0%|          | 0/1970 [00:00<?, ? examples/s]

NOT Subtopic: 1817
Train dataset: 1575
Subtopic test: 31
NOT Subtopic test: 364


In [8]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

In [9]:
from unsloth import to_sharegpt
from unsloth import standardize_sharegpt
from unsloth.chat_templates import get_chat_template

def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False, truncation=True,
        padding=True,).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

def format_dataset(dataset):
  dataset = to_sharegpt(
    dataset,
    merged_prompt="{Question}",
    output_column_name="Response",
    conversation_extension=3,  # Select more to handle longer conversations
  )
  dataset = standardize_sharegpt(dataset)

  dataset = dataset.map(formatting_prompts_func, batched = True)

  return dataset




In [10]:
d_train = format_dataset(d_train)

Merging columns:   0%|          | 0/1575 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/1575 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1575 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1575 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/1575 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/1575 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/1575 [00:00<?, ? examples/s]

Map:   0%|          | 0/1575 [00:00<?, ? examples/s]

In [11]:
d_train[10]["text"]

'<start_of_turn>user\n(\'In a scenario where a 32-year-old homeless woman is brought to the emergency department unresponsive with pinpoint pupils and low respiratory rate, intravenous administration of a drug promptly reverses these symptoms. Which receptor does the administered drug most strongly affect to produce this reversal of symptoms?\',)<end_of_turn>\n<start_of_turn>model\nThe administered drug is most strongly affecting the mu-opioid receptors to produce the reversal of symptoms in the case of an opioid overdose. The drug, likely naloxone, acts as an opioid antagonist by displacing opioids from these receptors, counteracting the effects like respiratory depression and pupil constriction.<end_of_turn>\n<start_of_turn>user\n("A 74-year-old man developed shortness of breath and left-sided back pain three days after hip arthroplasty for a right hip fracture. He reports sharp pain with deep breathing, and physical examination reveals a friction rub at the left lung base and swelli

# Training

In [12]:
print("---Training Model---")

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = d_train,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
          dataset_text_field="text",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,  
        per_device_eval_batch_size=4,
        eval_accumulation_steps=1,
        #num_train_epochs = 1,
        learning_rate = 2e-6,  # conservative default; increase to 2e-6 only if stable
        warmup_steps = 40,
        lr_scheduler_type = "cosine",
        max_steps=280,                    # fits in limited time
        logging_steps=5,
        optim="adamw_torch_fused",
        weight_decay=0.001,
        seed=13,
        report_to="none",
        packing=False,
        gradient_checkpointing=True,
    ),
)
tokenizer.model_max_length = 2048

---Training Model---
Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1575 [00:00<?, ? examples/s]

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=8):   0%|          | 0/1575 [00:00<?, ? examples/s]

In [14]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.531 GB of memory reserved.


In [15]:
print("Started Training")
trainer_stats = trainer.train()
print(trainer_stats)

Started Training


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,575 | Num Epochs = 2 | Total steps = 280
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 32,788,480 of 4,332,867,952 (0.76% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,3.862300
10,4.051500
15,3.580400
20,3.816900
25,3.631000
30,3.753800
35,3.480400
40,3.216000
45,3.134600
50,2.947000


TrainOutput(global_step=280, training_loss=1.9632875221116202, metrics={'train_runtime': 2867.4986, 'train_samples_per_second': 0.781, 'train_steps_per_second': 0.098, 'total_flos': 2.922192653356704e+16, 'train_loss': 1.9632875221116202, 'epoch': 1.4215873015873015})


In [16]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2867.4986 seconds used for training.
47.79 minutes used for training.
Peak reserved memory = 6.584 GB.
Peak reserved memory for training = 1.053 GB.
Peak reserved memory % of max memory = 44.665 %.
Peak reserved memory for training % of max memory = 7.143 %.


# Evaluation and saving

In [17]:
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only
from transformers import DataCollatorForLanguageModeling
from transformers import default_data_collator
from transformers import Trainer, TrainingArguments


def silly_eval(dataset):
    dataset = format_dataset(dataset)
    silly = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = d_train,
        eval_dataset = dataset,
        data_collator=default_data_collator,
        args=SFTConfig(
            packing=False,  # <-- crucial
            report_to="none",
            eval_steps = None,
            eval_strategy = "no"
        )
    )
    
    hf_trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=silly.data_collator,
        eval_dataset=silly.eval_dataset,
        args=TrainingArguments(
            report_to="none",
            fp16=True,
            per_device_eval_batch_size = 1,
            eval_accumulation_steps = 1,
        )
    )

    stats = hf_trainer.evaluate()
    print(stats)

In [18]:
print("Starting Evaluation")
silly_eval(d_test_NOTsubtopic)
silly_eval(d_test_subtopic)



Starting Evaluation


Merging columns:   0%|          | 0/364 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/364 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/364 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/364 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/31 [00:00<?, ? examples/s]

/tmp/ipykernel_20/2229970743.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  hf_trainer = Trainer(


{'eval_loss': 2.442798376083374, 'eval_model_preparation_time': 0.0097, 'eval_runtime': 27.2162, 'eval_samples_per_second': 1.139, 'eval_steps_per_second': 0.588}
